In [35]:
#functions

import pandas as pd
import numpy as np


###READS PDB FILE AND EXTRACT COORDINATES, ASSIGNING COORDINATES AS A NESTED LIST IN DICTIONARY WITH SUBUNIT AS KEY
###ONLY SUPPORTS SINGLE CHARACTER CHAIN IDS
###FILENAME: string. filename.
###RETURNS DICTIONARY CONTAINING ALL COORDINATES OF EACH SUBUNIT, SUBUNIT NAME AS DICTIONARY KEY
def readpdb(FILENAME):
    tempdict = {}
    firstatomline = True
    backboneatoms = ["C  ", "O  ", "N  ", "CA "]
    
    with open(FILENAME) as file_in:
        for line in file_in:
            if line.startswith(("ATOM","HETATM")):
                if firstatomline:
                    currentchain = line[21]
                    coord = line[31:54].split(" ")
                    coord = list(filter(None,coord))
                    for i in range(0,len(coord)):
                        coord[i] = float(coord[i])
                    tempdict[currentchain] = [coord]
                    firstatomline = False
                elif line[21] == currentchain:
                    coord = line[31:54].split(" ")
                    coord = list(filter(None,coord))
                    for i in range(0,len(coord)):
                        coord[i] = float(coord[i])
                    if currentchain in tempdict:
                        templist = tempdict[currentchain]
                        templist.append(coord)
                        tempdict.update(currentchain = templist)
                    else:
                        tempdict[currentchain] = [coord]
                else:
                    currentchain = line[21]
                    coord = line[31:54].split(" ")
                    coord = list(filter(None,coord))
                    for i in range(0,len(coord)):
                        coord[i] = float(coord[i])
                    if currentchain in tempdict:
                        templist = tempdict[currentchain]
                        templist.append(coord)
                        tempdict.update(currentchain = templist)
                    else:
                        tempdict[currentchain] = [coord]
    if "currentchain" in tempdict:
        tempdict.pop("currentchain")  ###idk why it makes this as a duplicate of the final chain sooooo
    return tempdict

###CALCULATES "CENTRE OF MASS" OF A SUBUNIT (DOES NOT FACTOR IN ATOM MASS)
###COORD: dictionary of subunits key with all atom coords as a nested list (i.e. output of readpdb())
###RETURNS CENTRE OF MASS AS A DATAFRAME (X,Y,Z COLUMNS; SUBUNIT ROWS)
def CoM(COORD):
    com = pd.DataFrame(columns=['x','y','z'])
    indexname = []
    for subunit in COORD:
        tempdf = pd.DataFrame(data=COORD[subunit])
        comxyz = [0,0,0]
        for m in range(0,len(tempdf.index)):
            comxyz[0] += tempdf.iat[m,0]
            comxyz[1] += tempdf.iat[m,1]
            comxyz[2] += tempdf.iat[m,2]
        for i in range(0,len(comxyz)):
            comxyz[i] /= len(tempdf.index)
        com.loc[len(com)] = comxyz
        indexname.append(subunit)
    com.index = indexname
    return com

###CALCULATES "CENTRE OF MASS" OF MULTIPLE SUBUNITS, BUT ASSUMING ALL SUBUNITS HAVE THE SAME "MASS" i.e. taking each subunit as mass = 1
###CoM: pd dataframe. Centres of mass of the subunits.
###SUBUNITS: list of string. A list of subunit names.
###RETURNS CENTRE OF MASS AS A LIST OF [x,y,z]
def CoMring(CoM,SUBUNITS):
    CoMs = []
    comxyz = [0,0,0]
    for subunit in SUBUNITS:
        CoMs.append(CoM.loc[subunit].tolist())
    for i in CoMs:
        comxyz[0] += i[0]
        comxyz[1] += i[1]
        comxyz[2] += i[2]
    for i in range(0,len(comxyz)):
        comxyz[i] /= len(SUBUNITS)
    return comxyz

###CALCULATES "CENTRE OF MASS" OF MULTIPLE SUBUNITS, TAKES INTO ACCOUNT NUMBER OF ATOMS IN SUBUNITS
###SUBUNITS: list of string. A list of subunit names.
###COORD: dictionary of subunits key with all atom coords as a nested list (i.e. output of readpdb())
###RETURNS CENTRE OF MASS AS A LIST OF [x,y,z]
def CoMmultisub(COORD,SUBUNITS):
    comxyz = [0,0,0]
    atomcount = 0
    for subunit in SUBUNITS:
        if subunit in COORD:
            tempdf = pd.DataFrame(data=COORD[subunit])
            for m in range(0,len(tempdf.index)):
                comxyz[0] += tempdf.iat[m,0]
                comxyz[1] += tempdf.iat[m,1]
                comxyz[2] += tempdf.iat[m,2]
            atomcount += len(tempdf.index)
        else:
            print("No subunit",subunit,"in pdb coords. Ignoring subunit")
    for i in range(0,len(comxyz)):
        comxyz[i] /= atomcount
    return comxyz

###CALCULATES LINE OF BEST FIT
###CoMs: nested list. Centre of masses in [[x,y,z],[x,y,z]...] format.
###RETURNS VECTOR (NOT NORMALISED) OF LINE OF BEST FIT AS A LIST OF [I,J,K]
def fitline(CoMs):
    df = pd.DataFrame(CoMs,columns=['x','y','z'])
    x = np.array(df["x"].tolist()) 
    y = np.array(df["y"].tolist()) 
    z = np.array(df["z"].tolist()) 
    data = np.concatenate((x[:, np.newaxis], 
                           y[:, np.newaxis], 
                           z[:, np.newaxis]), 
                          axis=1)
    datamean = data.mean(axis=0)
    uu, dd, vv = np.linalg.svd(data - datamean)
    linepts = vv[0] * np.mgrid[-10:10:2j][:, np.newaxis]
    vector = [linepts[1][0]-linepts[0][0],linepts[1][1]-linepts[0][1],linepts[1][2]-linepts[0][2]]
    adjlinepts = linepts + datamean 
    return vector, adjlinepts

###CALCULATES ANGLE BETWEEN 2 VECTORS
###V: VECTOR1. U: VECTOR2.
###RETURNS ANGLES IN DEGREES (ACUTE, OBTUSE)
def angle(V, U):
    angle = np.arccos(np.dot(V, U) / (np.linalg.norm(V) * np.linalg.norm(U)))
    return np.rad2deg(angle), np.rad2deg(np.pi-angle)

In [34]:
###ONLY READS BACKBONE ATOMS 
def readbbpdb(FILENAME):
    tempdict = {}
    firstatomline = True
    backboneatoms = ["C  ", "O  ", "N  ", "CA "]
    with open(FILENAME) as file_in:
        for line in file_in:
            if line.startswith(("ATOM","HETATM")):
                if line[13:16] in backboneatoms:
                    if firstatomline:
                        currentchain = line[21]
                        coord = line[31:54].split(" ")
                        coord = list(filter(None,coord))
                        for i in range(0,len(coord)):
                            coord[i] = float(coord[i])
                        tempdict[currentchain] = [coord]
                        firstatomline = False
                    elif line[21] == currentchain:
                        coord = line[31:54].split(" ")
                        coord = list(filter(None,coord))
                        for i in range(0,len(coord)):
                            coord[i] = float(coord[i])
                        if currentchain in tempdict:
                            templist = tempdict[currentchain]
                            templist.append(coord)
                            tempdict.update(currentchain = templist)
                        else:
                            tempdict[currentchain] = [coord]
                    else:
                        currentchain = line[21]
                        coord = line[31:54].split(" ")
                        coord = list(filter(None,coord))
                        for i in range(0,len(coord)):
                            coord[i] = float(coord[i])
                        if currentchain in tempdict:
                            templist = tempdict[currentchain]
                            templist.append(coord)
                            tempdict.update(currentchain = templist)
                        else:
                            tempdict[currentchain] = [coord]
    if "currentchain" in tempdict:
        tempdict.pop("currentchain")
    return tempdict


In [ ]:
#ApcE2 not trimmed gives the same angle (only keeping K,k:18-87, 149-153)
pdbcoords = readpdb("FR_APC_TRIMMEDAPCE.pdb")
CoMs = CoM(pdbcoords)

rings1 = [["A","B","C","D","E","F"],["G","H","I","J","K","L"],["M","N","O","P","Q","R"],["T","U","V","W","X","Y"]]
rings2 = [["a","b","c","d","e","f"],["g","h","i","j","k","l"],["m","n","o","p","q","r"],["t","u","v","w","x","y"]]
RingCoMs1 = []
RingCoMs2 = []
for i in rings1:
    RingCoMs1.append(CoMring(CoMs,i))
for i in rings2:
    RingCoMs2.append(CoMring(CoMs,i))

vector1, linepts1 = fitline(RingCoMs1)
vector2, linepts2 = fitline(RingCoMs2)

tiltangle1, tiltangle2 = angle(vector1, vector2)

In [ ]:
print("CoMs1:")
for i in RingCoMs1:
    print(i[0],",",i[1],",",i[2])

print("\nCoMs2:")
for i in RingCoMs2:
    print(i[0],",",i[1],",",i[2])

print("\npts for line of best fit 1:")
for i in linepts1:
    print(i[0],",",i[1],",",i[2])
    
print("\npts for line of best fit 2:")
for i in linepts2:
    print(i[0],",",i[1],",",i[2])
    
print("\nTilt is:")
print(tiltangle1, tiltangle2)

ApcE2 when full length is used gives an angle of 152.8259919745681 27.174008025431903.
ApcE2 when trimmed (keeping K,k:18-87, 149-253) gives an angle of 150.62253585648517 29.37746414351482.

In [2]:
#using commultisub instead of comring (does not assume all subunits have the same atom count)
pdbcoords = readpdb("FR-APC.pdb")
rings1 = [["A","B","C","D","E","F"],["G","H","I","J","K","L"],["M","N","O","P","Q","R"],["T","U","V","W","X","Y"]]
rings2 = [["a","b","c","d","e","f"],["g","h","i","j","k","l"],["m","n","o","p","q","r"],["t","u","v","w","x","y"]]
RingCoMs1 = []
RingCoMs2 = []
for i in rings1:
    RingCoMs1.append(CoMmultisub(pdbcoords,i))
for i in rings2:
    RingCoMs2.append(CoMmultisub(pdbcoords,i))
    
vector1, linepts1 = fitline(RingCoMs1)
vector2, linepts2 = fitline(RingCoMs2)

tiltangle1, tiltangle2 = angle(vector1, vector2)

In [3]:
print("CoMs1:")
for i in RingCoMs1:
    print(i[0],",",i[1],",",i[2])

print("\nCoMs2:")
for i in RingCoMs2:
    print(i[0],",",i[1],",",i[2])

print("\npts for line of best fit 1:")
for i in linepts1:
    print(i[0],",",i[1],",",i[2])
    
print("\npts for line of best fit 2:")
for i in linepts2:
    print(i[0],",",i[1],",",i[2])
    
print("\nTilt is:")
print(tiltangle1, tiltangle2)

CoMs1:
191.91621301020447 , 236.30616237244968 , 236.4068607142851
204.8256281360502 , 265.2346689678392 , 230.76074936858106
226.60817244063819 , 313.1148897106518 , 215.7823775788241
217.31063194173404 , 287.5351542463255 , 219.36725035765443

CoMs2:
314.18454604591795 , 269.79377729591846 , 236.40843048469276
301.27799941067514 , 240.86872798450975 , 230.76532396026263
279.4919118982744 , 192.9828871156099 , 215.78089918256111
288.79008050461533 , 218.56204551957396 , 219.36775523475038

pts for line of best fit 1:
206.11242100472865 , 266.76597730323266 , 228.12023675488405
214.2179017595848 , 284.3294603454004 , 223.03838225478836

pts for line of best fit 2:
299.9888159247048 , 239.33353155903689 , 228.12186341074417
291.8834530050366 , 221.77018739876917 , 223.0393410203893

Tilt is:
150.5583586858074 29.441641314192598


Using CoMsmultisub:
ApcE2 when full length is used gives an angle of 150.5583586858074 29.441641314192598.
ApcE2 when trimmed (keeping K,k:18-87, 149-253) gives an angle of 149.73346001304822 30.266539986951795.

In [4]:
#using commultisub instead of comring (does not assume all subunits have the same atom count), using 12-mer
pdbcoords = readpdb("FR-APC.pdb")
rings1 = [["A","B","C","D","E","F","G","H","I","J","K","L"],["M","N","O","P","Q","R","T","U","V","W","X","Y"]]
rings2 = [["a","b","c","d","e","f","g","h","i","j","k","l"],["m","n","o","p","q","r","t","u","v","w","x","y"]]
RingCoMs1 = []
RingCoMs2 = []
for i in rings1:
    RingCoMs1.append(CoMmultisub(pdbcoords,i))
for i in rings2:
    RingCoMs2.append(CoMmultisub(pdbcoords,i))
    
vector1 = [RingCoMs1[1][0]-RingCoMs1[0][0],RingCoMs1[1][1]-RingCoMs1[0][1],RingCoMs1[1][2]-RingCoMs1[0][2]]
vector2 = [RingCoMs2[1][0]-RingCoMs2[0][0],RingCoMs2[1][1]-RingCoMs2[0][1],RingCoMs1[1][2]-RingCoMs2[0][2]]

tiltangle1, tiltangle2 = angle(vector1, vector2)

In [5]:
print("CoMs1:")
for i in RingCoMs1:
    print(i[0],",",i[1],",",i[2])

print("\nCoMs2:")
for i in RingCoMs2:
    print(i[0],",",i[1],",",i[2])
    
print("\nTilt is:")
print(tiltangle1, tiltangle2)

CoMs1:
199.69276402272118 , 253.73251399736301 , 233.0056785170919
221.96483723044847 , 300.3399750584552 , 217.57271836840727

CoMs2:
306.40972299421975 , 252.36950831727302 , 233.0090583730599
284.13556079501285 , 205.7575135749546 , 217.5722304494673

Tilt is:
146.7293136831558 33.27068631684422


Taking CoM of 12-mer:
ApcE2 when full length is used gives an angle of 146.7293136831558 33.27068631684422.
ApcE2 when trimmed (keeping K,k:18-87, 149-253) gives an angle of 146.4768883742232 33.52311162577679.

In [36]:
#only reading backbone atoms
pdbcoords = readbbpdb("FR_APC_TRIMMEDAPCE.pdb")
rings1 = [["A","B","C","D","E","F"],["G","H","I","J","K","L"],["M","N","O","P","Q","R"],["T","U","V","W","X","Y"]]
rings2 = [["a","b","c","d","e","f"],["g","h","i","j","k","l"],["m","n","o","p","q","r"],["t","u","v","w","x","y"]]
RingCoMs1 = []
RingCoMs2 = []
for i in rings1:
    RingCoMs1.append(CoMmultisub(pdbcoords,i))
for i in rings2:
    RingCoMs2.append(CoMmultisub(pdbcoords,i))
    
vector1, linepts1 = fitline(RingCoMs1)
vector2, linepts2 = fitline(RingCoMs2)

tiltangle1, tiltangle2 = angle(vector1, vector2)

In [40]:
print("CoMs1:")
for i in RingCoMs1:
    print(i[0],",",i[1],",",i[2])

print("\nCoMs2:")
for i in RingCoMs2:
    print(i[0],",",i[1],",",i[2])

print("\npts for line of best fit 1:")
for i in linepts1:
    print(i[0],",",i[1],",",i[2])
    
print("\npts for line of best fit 2:")
for i in linepts2:
    print(i[0],",",i[1],",",i[2])

    
print("\nVector is:")
print(vector1,vector2)
print("\nTilt is:")
print(tiltangle1, tiltangle2)

CoMs1:
191.8898135942325 , 236.13458496395435 , 236.43664212152387
202.64333954081644 , 260.9146094387749 , 233.5559778061229
226.6721548742132 , 313.28410875262114 , 215.76984198113138
217.24063249211403 , 287.363645373292 , 219.38621161934836

CoMs2:
314.2128934088571 , 269.96401004119423 , 236.43609938208002
303.4546525510198 , 245.1869459183677 , 233.55604107142824
279.4282329664571 , 192.81452777777776 , 215.76954821803042
288.86122739221815 , 218.73442718191376 , 219.3890073606727

pts for line of best fit 1:
205.5611347269044 , 265.66326192274784 , 228.9025029743763
213.66183552378368 , 283.1852123415734 , 223.67183378968699

pts for line of best fit 2:
300.53964025557104 , 240.4359812414811 , 228.90285450706617
292.43886290370494 , 222.9139742181457 , 223.67249350903953

Vector is:
[8.10070079687928, 17.521950418825558, -5.230669184689283] [-8.100777351866054, -17.52200702333542, -5.230360998026663]

Tilt is:
149.6787714048963 30.32122859510368
